In [8]:
import urllib
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import ssl
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pytesseract
import cv2 
import numpy as np
import requests
from PIL import Image
from firebase import firebase
import multiprocessing as mp

In [9]:
DRIVER = '/home/kinetic/chromedriver_linux64/chromedriver'
#firebase
fb = firebase.FirebaseApplication('https://result3rdsempy.firebaseio.com/',None)



In [ ]:
#script to check enrollment numbers which are not uploaded in firebase
left = list(range(1,127))
for key in fb.get('result4thsempy/','result'):
    no = int(key)%1000
    left.remove(no)
print(len(left),left)   

In [14]:
#Function for MultiProcessing
DRIVER = '/home/kinetic/chromedriver_linux64/chromedriver'


def finalResult(no):
    no=no[0]
    
    #firebase
    fb = firebase.FirebaseApplication('https://result3rdsempy.firebaseio.com/',None)
    driver = webdriver.Chrome(DRIVER)
    #driver.minimize_window()
    driver.get("http://gturesults.in/Default.aspx?ext=S2020&rof=2398"); 
    #enrollment = 180170107000+no#regular
    enrollment = 190173107000+no#D2ds
    #take SS and crop
    screenshot = driver.save_screenshot(str(no)+'.png')
    #driver.minimize_window()
    img=Image.open(str(no)+'.png')
    bounding_box = (219,220,330,250)
    captcha=img.crop(bounding_box)

    #driver.minimize_window()
    #find elements in from and fill values
    #batch = driver.find_element_by_id('ddlbatch')
    #batch.send_keys('.....BE SEM 3 - Regular (DEC 2019)')
    enroll = driver.find_element_by_id('txtenroll')
    enroll.clear()
    enroll.send_keys(str(enrollment))
    captcha_text = text_captcha(captcha)
    capEnter = driver.find_element_by_id('CodeNumberTextBox')
    capEnter.clear()
    capEnter.send_keys(captcha_text)
    search = driver.find_element_by_id('btnSearch')
    search.send_keys(Keys.ENTER)

    #store html in content
    content=driver.page_source
    driver.quit()


    soup =BeautifulSoup(content,'html5lib')
    msg = soup.find('span',attrs={'id':'lblmsg'}).text.strip()
    if msg == "Oppssss! Data not available.":
        print(enrollment,'Data is not available')
        #sys.stdout.flush()
        return 0
    elif msg == "ERROR: Incorrect captcha code, try again.":
        print(enrollment,'Invaid Captcha')
        #sys.stdout.flush()
        return finalResult([no])

    Enroll = soup.find('span',attrs={'id':'lblEnrollmentNo'}).text
    print('Enrollment:',Enroll)
    #sys.stdout.flush()
    Name = soup.find('span',attrs={'id':'lblName'}).text
    print("Name:",Name)
    if Name == '------------':
        print(enrollment,'Unknown Error!')
        return finalResult([no])
    #sys.stdout.flush()
    fb.put('result4thsempy/result/'+str(Enroll),'Name',Name)

    tables = soup.find_all('table',attrs={'class':'Rgrid'})

    #grading table   
    innersoup =BeautifulSoup(str(tables[1]),'html5lib')
    allTd=innersoup.find_all('td')
    subcode_ind=0
    sub_ind=1
    finalgrade_ind=15
    data={}
    while(True):
        try:
            sub_code=allTd[subcode_ind].text.strip()
            sub_name=allTd[sub_ind].text.strip()
            grade=allTd[finalgrade_ind].text.strip()
            #print(sub_code,sub_name,grade)
            #sys.stdout.flush()
            data[sub_name]={'sub_code':sub_code,'grade':grade}  
        except:
            fb.put('result4thsempy/result/'+str(Enroll),'Grades',data)
            break
        subcode_ind+=16
        sub_ind+=16
        finalgrade_ind+=16

    #spi,cpi,backlogs
    spisoup =BeautifulSoup(str(tables[2]),'html5lib')
    other=spisoup.find_all('td')
    current_sem_backlog=other[0].text.strip().split(':')[1].strip()
    total_backlog=other[1].text.strip().split(':')[1].strip()
    spi = other[2].text.strip().split(':')[1].strip()
    cpi=other[3].text.strip().split(':')[1].strip()
    fb.put('result4thsempy/result/'+str(Enroll),'current_sem_backlog',current_sem_backlog)
    fb.put('result4thsempy/result/'+str(Enroll),'total_backlog',total_backlog)
    fb.put('result4thsempy/result/'+str(Enroll),'SPI',spi)
    fb.put('result4thsempy/result/'+str(Enroll),'CPI',cpi)
    print(enrollment,'Data is Successfully uploaded')
    return 1


In [11]:
def binirize(image_to_transform,threshold):
    image = image_to_transform.convert("L")
    w = image.width
    h = image.height
    for x in range(w):
        for y in range(h):
            if image.getpixel((x,y)) < threshold:
                image.putpixel((x,y),0)
            else:
                image.putpixel((x,y),255)
    return image
def text_captcha(captcha_image):
    new_size=(captcha_image.width*3,captcha_image.height*3)
    bigcaptcha = captcha_image.resize(new_size)
    img=binirize(bigcaptcha,160)
    w = img.width
    h = img.height
    #remove black line from captcha(which lie on pixel 52 to 58 in vertical direction)
    for y in range(52,58):
        for x in range(0,w):
            if img.getpixel((x,y-1)) < 100:
                continue
            img.putpixel((x,y),255)
   # display(img)        
    return pytesseract.image_to_string(img)

In [24]:
#MultiProcessing
workers = mp.Pool(4)
for eachtest in workers.map(finalResult,[[no] for no in range(5,6)]):
    print(eachtest)

Enrollment: 190173107005
Name: CHAUDHARI DEVAM VIJAYKUMAR
190173107005 Data is Successfully uploaded
1


In [29]:
#sorting according SPI
#result3rdsempy/result/180170107062/SPI
import csv
Dict = fb.get('result4thsempy/','result')
with open('4thsemResult.csv','w') as f:
    csvwriter = csv.writer(f)
   
    for enroll in Dict:
        #print(enroll)
        row = []
        row.append(enroll)
        row.append(Dict[enroll]['Name'])
        row.append(Dict[enroll]['Grades']['Computer Organization & Architecture']['grade'])
        row.append(Dict[enroll]['Grades']['Design Engineering 1 B']['grade'])
        row.append(Dict[enroll]['Grades']['Discrete Mathematics']['grade'])
        row.append(Dict[enroll]['Grades']['Object Oriented Programming -I']['grade'])
        row.append(Dict[enroll]['Grades']['Operating System']['grade'])
        row.append(Dict[enroll]['Grades']['Principles of Economics and Management']['grade'])
        row.append(float(Dict[enroll]['SPI']))
        row.append(float(Dict[enroll]['CPI']))
        row.append(float(Dict[enroll]['current_sem_backlog']))
        row.append(float(Dict[enroll]['total_backlog']))
        csvwriter.writerow(row)
        


In [27]:
name_SPI.sort(key = lambda x:x[1],reverse=True)    

In [28]:
Rank=1
for i in name_SPI:
    print(Rank,"):",i[0],"--",i[1])
    Rank+=1

1 ): PATEL MAHISHA HARSHAD -- 9.65
2 ): DOSHI ANANT BHAVESHKUMAR -- 9.61
3 ): UKANI KUNJ BHARATKUMAR -- 9.61
4 ): PATEL SNEHEE VINODKUMAR -- 9.35
5 ): SHAIKH KHUSHNOOD JAVEDAKHTAR -- 9.35
6 ): LAD DEEP PRAVINBHAI -- 9.26
7 ): PATEL VISHWA BANSILAL -- 9.26
8 ): TANMAY DINESHKUMAR SHAH -- 9.26
9 ): TRIVEDI TIRTHKUMAR NEHALKUMAR -- 9.26
10 ): SOLIYA SHIVAM RAMESHBHAI -- 9.22
11 ): VAZIRANI GUNJAN RAJESH -- 9.22
12 ): BHATIYA RIYA ASHISH -- 9.17
13 ): MAHESHWARI PAYAL MADANBHAI -- 9.17
14 ): JHA SHRISTI HARLAL -- 9.13
15 ): THAKKAR AAYUSH BHARATBHAI -- 9.13
16 ): SARVESH DINESH PRAJAPATI -- 9.09
17 ): GOYANI KEVALKUMAR SAVAJIBHAI -- 9.04
18 ): PATEL YASH RAKESHBHAI -- 9.04
19 ): SHAH RIYA DHARMESHBHAI -- 9.04
20 ): PRAJAPATI MANSI GAUTAMBHAI -- 9.04
21 ): DESAI DHWANI BHAVIKBHAI -- 9.0
22 ): VEKARIYA HARDIKKUMAR MANUBHAI -- 9.0
23 ): PANCHAL SMITKUMAR KALPESHBHAI -- 8.91
24 ): RAJPUROHIT YOGESHSINGH OMSINGH -- 8.91
25 ): MISTRY VISHALKUMAR MANOJBHAI -- 8.87
26 ): PATEL YASH NARESHBHAI -- 8